In [1]:
!git clone https://github.com/CalvQ/RAGnaRec.git

Cloning into 'RAGnaRec'...
remote: Enumerating objects: 372, done.
remote: Counting objects: 100% (111/111), done.
remote: Compressing objects: 100% (77/77), done.
remote: Total 372 (delta 57), reused 82 (delta 31), pack-reused 261 (from 1)
Receiving objects: 100% (372/372), 15.69 MiB | 9.45 MiB/s, done.
Resolving deltas: 100% (204/204), done.
Updating files: 100% (74/74), done.


In [2]:
%cd RAGnaRec

/content/RAGnaRec


In [3]:
!pip install --quiet --upgrade langchain langchain-community langchain-chroma
!pip install -U langchain langchain-openai
!pip install sentence-transformers langchain chromadb
!pip install seaborn
!pip install bertopic
!pip install pyarrow fastparquet huggingface_hub matplotlib seaborn datasets

# Import packages
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
from scipy import sparse
from sklearn.preprocessing import normalize
from sklearn.manifold import TSNE
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.decomposition import PCA
import random
import torch

import getpass
import os

from langchain.vectorstores import Chroma
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.document_loaders import DataFrameLoader
from transformers import pipeline

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 5.6 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 28.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 80.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 628.3/628.3 kB 42.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 88.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 94.8/94.8 kB 9.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 410.6/410.6 kB 29.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 278.6/278.6 kB 21.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 64.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.5/49.5 kB 4.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 93.2/93.2 kB 8.4 MB/s eta 0:00:00


In [4]:
from bertopic import BERTopic
import json

In [ ]:
topic_model = BERTopic.load("../E2E")

# Part 1: Get the User Review

In [5]:
review = input("Enter Your Review: ")

Enter Your Review: I really enjoyed eating at Noodle Head in Pittsburgh. The Pad Thai was amazing. My only issue was the long wait time. Can you find me similar restaurants?


In [6]:
review

'I really enjoyed eating at Noodle Head in Pittsburgh. The Pad Thai was amazing. My only issue was the long wait time. Can you find me similar restaurants?'

# Part 2: Assign the Input Review to its Topic

In [8]:
def assign_topic(review, topic_model):
    topics, probabilities = topic_model.transform([review])
    return {
        'topic': topics[0],
        'probability': probabilities[0],
        'keywords': topic_model.get_topic(topics[0]) if topics[0] != -1 else None
    }

assign_topic(review, topic_model)

{'topic': 6,
 'probability': 0.6715214,
 'keywords': [['thai', 0.029264627200730444],
  ['chinese', 0.02893741835325305],
  ['rice', 0.020044305212758705],
  ['noodle', 0.01845541880352594],
  ['food', 0.01648652493703186],
  ['dish', 0.01572518655254208],
  ['chicken', 0.015428031701884065],
  ['curry', 0.014553300168216968],
  ['soup', 0.014071337657354379],
  ['pho', 0.01348777691219665]]}

### If unclustered, go through a word search

In [9]:
def find_topic(review, topics_file_path='E2E/topics.json'):
    # Load topics dictionary
    with open(topics_file_path, 'r') as f:
        topics_dict = json.load(f)

    topics_dict = topics_dict['topic_representations']

    # Convert review text to lowercase and split into words
    review_words = set(review.lower().split())

    max_probability = 0
    best_topic = None
    best_word = None
    best_prob = 0

    # For each topic and its list of [word, probability] pairs
    for topic_id, word_pairs in topics_dict.items():
        # Extract words and probabilities for this topic
        for word, prob in word_pairs:
            if word.lower() in review_words and prob > max_probability:
                max_probability = prob
                best_topic = int(topic_id)
                best_word = word
                best_prob = prob

    if best_topic is not None:
        return best_topic, (best_word, best_prob)
    return -1, None


In [10]:
find_topic(review)

(6, ('thai', 0.029264627200730444))

# Part 3: RAG Pipeline

In [11]:
from rag_pipeline.rag import main

In [12]:
response = main(review)
print(response)

Please enter your Hugging Face token:
··········
Successfully logged in to Hugging Face.


/content/RAGnaRec/rag_pipeline/retrieval.py:31: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embeddings = HuggingFaceEmbeddings(model_name = embedding_model)


config.json:   0%|          | 0.00/838 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/24.2k [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/4.99G [00:00<?, ?B/s]

...

model.safetensors:   0%|          | 0.00/268M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

/content/RAGnaRec/rag_pipeline/generator.py:74: LangChainDeprecationWarning: The method `BaseRetriever.get_relevant_documents` was deprecated in langchain-core 0.1.46 and will be removed in 1.0. Use :meth:`~invoke` instead.
  retrieved_docs = retriever.get_relevant_documents(user_review)


Based on your review, it sounds like you enjoyed the Pad Thai at Noodle Head in Pittsburgh.  Since you're looking for similar places, here are a few recommendations:

**For similar Pad Thai:**

* **Thai Basil:** This restaurant is known for its authentic Thai dishes and has a 4.7-star rating on Yelp.  Reviewers rave about their Pad See Ew and Green Curry.
* **Bangkok Thai Cuisine:**  This spot has a 4.5-star rating and is highly praised for its flavorful Pad Thai and other Thai classics. 

Let me know if you'd like more recommendations!


In [13]:
review = input("Enter Your Review: ")


Enter Your Review: The hotel I stayed at, Urban Retreat, had bedbugs and rude staff. What other hotels should I avoid?


In [14]:
review

'The hotel I stayed at, Urban Retreat, had bedbugs and rude staff. What other hotels should I avoid?'

In [15]:
response = main(review)
print(response)

Based on your review, it seems you're looking to avoid hotels with bedbugs and rude staff.  Here are a few places you might want to steer clear of:

* **Urban Retreat:**  This hotel has been flagged for bedbugs and has received negative feedback about its staff. 
* **[Hotel Name]**:  [Hotel Name] has received multiple negative reviews mentioning bedbugs and poor customer service. 


It's important to note that these are just a few examples, and there are many other hotels out there that might not meet your expectations.
